In [ ]:
import numpy as np
import math as math
import random as random
import time as time
import os
from scipy.optimize import root as root
from scipy.linalg import cholesky

class GKROM():
    def __init__(self, M, path = None, folder_name = None, simuSize = None, match4 = False, 
                 fixedOmega = True, initStyle = 'uniform', maxT_col2 = 30, maxT_incr = 4, edf = 2, 
                 xtol = None, maxiter = None, max_RO = 20): 
        '''
        Parameters
        ----------
        M : the target sample
        path :  the path to save the results
        folder_name : the folder_name for saving the results 
        simuSize : the number of observations in the simulation
        match4 :  matching the first three moments if False, else matching the first four moment
        fixedOmega : using the fixed rotation matrix in the paper if True, or using the random normal rotation matrix 
        maxT_col2 : the maximun time allowed for solving the second column
        maxT_incr : incremental multiplier for the maximum time allowed for solving the next column
        edf  : extra degrees of freedom of the coefficent vector for each column
        xtol : tolerance for termination when finding a real solution of the equation system
        maxiter : the maximun iterations for finding a real solution of the equation system
        max_RO :  the maximun time allowed for finding a proper orthognoal vectors
        '''  
        
        # set path to save the results
        if (path != None) and (folder_name == None):
            folder_name = 'simulated_path'
            
        if path != None:
            try:
                os.mkdir(path + folder_name + '/')
            except:
                pass
            self.path = path + folder_name + '/'            
        else:
            self.path = path
        
        # compute mean, cov and standardize the target sample
        
        self.mu = np.mean(M, axis = 0)   # mean     
        cov = np.cov(M.T)                # covariance
        self.A = cholesky(cov)          
        A_inv = np.linalg.inv(self.A)
        self.L_o = (M - self.mu).dot(A_inv)  # standarise sample
        
        # obtain the size of simulation
        
        self.m = [M.shape[0] if simuSize == None else simuSize][0]
        
        # obtain the target skewness
        
        self.b = self.Kolloskewness(self.L_o)
        
        # set parameters for the GKROM algorithm
        
        self.match4 = match4
        
        self.fixedOmega = fixedOmega
        
        # options used only for targeting Kollo kurtosis, these options are ignored when targeting only Kollo skewness
        
        self.initStyle = initStyle
        
        self.maxT_col2 = maxT_col2
        
        self.maxT_incr = maxT_incr
        
        self.edf = edf
        
        self.xtol = xtol
        
        self.maxiter = maxiter
        
        self.max_RO = max_RO
        
 

        
    def run(self):
        if not self.match4:  
            # obtain simulated standardised sample using GKROM3 which only targets Kollo skewness
            self.stdized_output = self.target_Kskewness(self.m, self.b, self.L_o, self.fixedOmega)
        else:
            # obtain simulated standardised sample using GKROM4 which targets Kollo skewness and kurtosis
            self.K = self.Kollokurtosis(self.L_o)   # obtain the target Kollo kurtosis
            self.stdized_output = self.target_Kskewkurt(self.m, self.b, self.K, self.L_o, self.fixedOmega, 
                                                        self.initStyle, self.maxT_col2, self.maxT_incr, self.edf, 
                                                        self.xtol, self.maxiter, self.max_RO)
        
        # obtain the simulation to match the mean and cov
        self.output = self.stdized_output.dot(self.A) + self.mu
        
        # save the simulation if necessary
        if self.path != None:
            np.savetxt(self.path + "Simulated_sample_" + str(int(time.time()*100000)) + ".csv", self.output, delimiter=",")

            
            
###########################################################
######################## Main functions ###################
###########################################################
    def target_Kskewness(self, m, b, L_o, fixedOmega): 
        '''
        This function is to produce a standardized sample L_mn targeting kollo skewness b
        
        Parameters
        ----------
        m    : the number of observations in simulations
        b    : the target Kollo skewness vetcor
        L_o  : the target sample after standardization. The Kollo skewness of the target sample should be equal to b.
        fixedOmega : using the fixed rotation matrix in the paper if True, or using the random normal rotation matrix 
        
        Return
        ----------
        L    : the standardised simulated sample, i.e. scaled L matrix
        '''
        
        # step 1: check whether b is within the maximum attainable norms of Kollo skewness vectors (see eq 20, Hanke et al. 2017)
        n = b.shape[0]
        assert not np.linalg.norm(b) >  n * (m - 2) / math.sqrt(m - 1)

        # step 2: produce omega matrix and get rotated skewness and rotated sample
        omega = self.RandomOmega(n, fixedOmega)   # obtain a rotation matrix 
        bstar = n**(-1) * omega.T.dot(b)     # obtain the rotated skewness
        RD = np.dot(L_o, omega)              # obtain the rotated target sample

        # step 3: product the first column for L_star based on target sample     
        L_1 = self.get_first_column_Skew(m, bstar[0], RD[:,0])
        
        #step 4: solve the remaining columns
        Ones = np.ones((m,1))
        u = L_1.reshape((m,1))
        
        i = 0
        while i < n-1:
            # generate orthogonal vectors
            v1 = (L_1**2).reshape((m,1))
            v2 = np.array(random.choices(RD[:, i+1], k = m)).reshape((m,1))
            v = np.concatenate((Ones, u, v1, v2), 1)
            
            # orthogonalise these vectors
            v = self.GS_k(v, i+2)
            Z1 = v[:, i+2]
            Z2 = v[:, i+3]
            
            # check the criterion (14) in the paper to see whether the system has a real solution   
            a = np.dot(L_1**2, Z1) / m
            b = np.dot(L_1**2, Z2) / m
            delta = a**2 + b**2 - bstar[i+1]

            if delta < 0:
                continue
            
            # obtain the values of the coefficients 
            g1 = (a * bstar[i+1] + b * np.sqrt(delta))/(a**2 + b**2)
            g2 = (b*bstar[i+1] - a*np.sqrt(delta))/(a**2 + b**2)
            
            # obtain the next column of scaled L matrix and concatenate it with the previous columns
            Z3 = (g1*Z1 + g2*Z2).reshape((m,1))
            u  = np.concatenate((u,Z3),1)
            i += 1
        
        L_star = u  # obtain the scaled L matrix

        # step 5: rotated back to obtain the standardised simulated sample
        L = L_star.dot(omega.T)

        return L

    
    def target_Kskewkurt(self, m, b, kurt, L_o, fixedOmega, initStyle, maxT_col2, maxT_incr, edf, xtol, maxiter, max_RO): 
        '''
        This function is to produce a standardized sample L_mn targeting Kollo skewness b and Kollo kurtosis kurt
        
        Parameters
        ----------
        m    : the number of observations in simulations
        b    : the target Kollo skewness vetcor
        kurt : the target Kollo kurtosis matrix
        L_o : the target sample after standardization
        fixedOmega : using the fixed rotation matrix in the paper if True, or using the random normal rotation matrix 
        maxT_col2 : the maximun time allowed for solving the second column
        maxT_incr : incremental multiplier for the maximum time allowed for solving the next column
        edf  : extra degrees of freedom of the coefficent vector for each column
        xtol : tolerance for termination when finding a real solution of the equation system
        maxiter : the maximun iterations for finding a real solution of the equation system
        max_RO :  the maximun time allowed for finding a proper orthognoal vectors
        
        Return
        ----------
        L    : the L_mn matrix 
        '''        
        
        # step 1: check whether b is within the maximum attainable norms of Kollo skewness vectors (see eq 20, Hanke et al. 2017)
        n = b.shape[0]
        assert not np.linalg.norm(b) >  n * (m-2) / np.sqrt(m-1)

        start_core = time.time()
        flag = 0
        while flag == 0:
            # step 2: produce omega matrix and get rotated skewness, kurtosis and data
            omega = self.RandomOmega(n, fixedOmega)   # obtain a rotation matrix

            Kstar = n**(-1) * np.dot(omega.T,kurt).dot(omega) # obtain the rotated skewness
            bstar = n**(-1) * omega.T.dot(b)   # obtain the rotated kurtosis
            RD = np.dot(L_o, omega)          # obtain the rotated target sample
            
            # step 3: product the first column for L_star
            L_1 = self.get_first_column_SkewKurt(m, bstar[0], Kstar[0,0], RD[:,0])
            print('Column 1 completed...')
            
            # step 4: solve the rest columns
            u = L_1.reshape((m,1))   # to save scaled L matrix
            
            i = 0
            maxT = maxT_col2 * (i + 1)   # update the maximun time allowed for solving each column
            while i < n-1:
                flag = 0
                start = time.time()
                Kstar_i= Kstar[i+1, 0:(i+2)].reshape((i+2,1))   # obtain the target vector in Kollo kurtosis for solving this column 
                
                while flag == 0:
                    # generate a set of orthogonal vectors 
                    no = (i+4) + edf     # set the number of orthognal vectors
                    [theta, A, B, D] = self.RandomOrthogonal(m, no, L_1, u, bstar[i+1], Kstar_i, RD[:,i+1], max_RO) 
                                   
                    # define the set of equations to solve for the coefficients 
                    # this def is set for the root() function later. it differs for different columns
                    def f1(x):
                        a = self.SolveSkewKurt( x, no, A, B, D, theta, bstar[i+1], Kstar_i)
                        return a
                        
                    # set initial values for root() function to solve the above set of equations: 
                    # !!! the speed of finding a solution is largely dependent on the initial values
                    if initStyle == 'equal':
                        init = np.array([1 / no] * no)
                    elif initStyle == 'uniform':
                        init = np.random.uniform(-1, 1, no)
                    init /= np.sqrt(init.dot(init))
                    
                    # using root() to solve for the coefficient vector
                    coefficient = root(f1, init, method = 'broyden1', options = {'xtol': xtol,'maxiter': maxiter})
                        
                    Time = time.time() - start
                    
                    if coefficient.success:   # if find a real solution of the coefficient vector
                        flag = 1
                        maxT = maxT_incr * maxT
                        coefficient = np.array(coefficient.x).reshape(no, 1)
                        
                        # obtain the next column of the scaled L matrix and concatenate it with the previous columns
                        Z3 = theta.dot(coefficient)
                        u = np.concatenate((u, Z3), 1)
                        print("Column " + str(i+2) + ' completed...')
                        i += 1
                    elif (Time > maxT): # if the time spent on solving for the coefficients exceed the limit maxT, break 
                        break           # break out of the loop and start from column 1 again.
                if (flag == 0) and (Time > maxT): # if the time spent on solving for the coefficients exceed the limit maxT, break 
                    break                         # break out of the loop and start from column 1 again.

        L_star = u.copy()    # obtain the scaled L matrix
        
        # step 5: get L form L_star
        L = L_star.dot(omega.T)    # rotated back to obtain the standardised simulated sample
        
        print('Core code running time: ' + str(round(time.time() - start_core, 2)) + ' seconds')
        return L  

###################################################################
######################## Intermediate functions ###################
###################################################################

#################### create orthonormal basis and create equation system for coefficients ####################
#################### for targeting kurtosis. Targeting skewness does not need these because  #################
#################### analytical solution is provided                                         #################
    def RandomOrthogonal(self, m, no, L_1, u, b, K, RD, max_RO):  
        '''
        This function is to generate orthonormal vectors and check some necessary conditions

        Parameters
        ----------
        m  : the size of simulations
        no : number of vectors needed
        L_1: the first column of scaled L matrix 
        u  : the prevous columns of scaled L matrix
        b  : the k-th value in rotated Kollo skenwess
        K  : a vector from the k-th column of rotated Kollo kurtosis matrix
        RD : the k-th column of rotated target sample (if not boostrapped, this is a random normal variable)
        max_RO: the time limit on generating a good random orthogonal matrix
        
        Return
        ---------
        theta : orthogonal vectors 
        A, B, D : parameters in the equations system
        
        '''
        RD = RD.flatten()
        nc, Ones = u.shape[1], np.ones((m,1))   # nc: the number of previous columns
        
        flag = 0
        Start_ = time.time()
        while flag == 0:  
            # generating random vectors
            # the speed of finding a solution is largely dependent on the random vecotrs
            v2 = np.array(random.choices(RD, k = (no-1) * m)).reshape((m, no-1))
            vs = (np.sign(L_1) * (L_1**2)).reshape((m, 1))
            vs = np.concatenate((Ones, u, vs, v2), 1)
    
            # orthogonalized these vectors
            vs = self.GS_k(vs, nc + 1)
            theta = vs[:, (nc + 1):]
            
            # compute the parameters of linear equations
            A = (np.array(no * [L_1 * L_1]).reshape(no, m).T) * theta   # parameters in linear equation for matching skewness
            D = (np.array(nc * [L_1 * L_1]).reshape(nc, m).T) * u       # parameters in linear equation for matching kurtosis
            
            # compute the distances between the linear equations and the quadratic equation (a circle with r=1) 
            Matrix = np.concatenate((m**(-1) * np.dot(Ones.T, A), m**(-1) * (np.dot(D.T, theta))), 0)
            column = np.concatenate((b.reshape((1,1)), K[0:nc, :]),0).reshape(nc + 1)
            distances = np.linalg.norm(Matrix, axis = 1)**(-1) * column
            
            # regenerate those orthogonal vectors who leave the linear equations without intersection with the quadratic equation (a circle with r=1) 
            # To control for time, if the process exceed the allowed time, we stop and give output directly regardless
            if any(abs(distances) <= 1) or ((time.time() - Start_) > max_RO):
                flag = 1 
        
        # compute the parameters of the quadtatic equation for matching kurtosis    
        B = (np.array(no * [L_1]).reshape(no, m).T) * theta 
    
        return [theta, A, B, D] 

    
    def SolveSkewKurt(self, x, no, A, B, D, theta, b, K):
        '''
        This function is to create an equation system for coefficients vectors
        
        Parameters
        ----------
        x  : the unkowns in the equation system
        no : the number of orthonal vectors
        A,B,D: the parameters in the equations
        theta: the orthonomal vectors
        b  : the k-th value in rotated Kollo skenwess
        K  : a vector from the k-th column of rotated Kollo kurtosis matrix
        ''' 
        m, nc = np.shape(theta)[0], D.shape[1]
        Ones = np.ones((m,1))
        
        # define the equations for matching skewness and kurtosis
        y = np.zeros((no,1))   # y is a column vector
        y[0] = np.dot(x.T, x) - 1
        y[1] = m**(-1) *  np.dot(Ones.T, A).dot(x) - b
        y[2] = m**(-1) *  np.dot(np.dot(B,x).T, np.dot(B,x)) - K[-1,0]
        y[3:3 + nc] = m**(-1) * (np.dot(D.T,theta).dot(x)).reshape((nc,1)) - K[0:nc,:]
        y = y.ravel()

        return y

###################################################################

######################## get first column #########################
    def get_first_column_Skew(self, m , b_1, Data):

        '''
        This function is to produce the first column for a scaled L_star matrix
        
        Parameters
        ----------
        m    : the number of observations in simulations
        b_1  : the first element in rotated Kollo skewness
        Data : the rotated target sample
        
        Return
        ----------
        L_1    : the first column for a scaled L_star
        '''        

        Ones = np.ones((m,1))
        flag = 0
        while flag == 0:
            # generate random vectors
            v1 = np.array(random.choices(Data, k = m)).reshape((m,1))
            v2 = np.array(random.choices(Data, k = m)).reshape((m,1))
            v3 = np.array(random.choices(Data, k = m)).reshape((m,1))
            v = np.concatenate((Ones,v1,v2,v3),1)
            
            # orthogonalise these vectors
            v = self.GS_k(v,1)
            Z1 = v[:,1]
            Z2 = v[:,2]
            Z3 = v[:,3]
            
            # define the equations for the first column
            def f1(x):
                y = np.zeros((3,1))   # y is a column vector
                y[0] = x[0]**2 + x[1]**2 + x[2]**2  - 1
                y[1] = np.mean((x[0] * Z1 + x[1] * Z2)**3 + x[2] * Z3) - b_1
                return y
            
            # solve equations to get the coefficients
            x_init = np.array([1,0,0])
            x_ret = root(f1, x_init, method = 'broyden2') 
            if x_ret.success:
                flag = 1
        
        # obtain the solution of the coefficents
        c = x_ret.x
        
        # obtain the first column of scaled L matrix
        L_1 = c[0] * Z1 + c[1] * Z2 + c[2] * Z3

        return L_1    

    def get_first_column_SkewKurt(self, m , b_1, k11, Data):
        '''
        This function is to produce the first column for a scaled L_star matrix
        
        Parameters
        ----------
        m    : the number of observations in simulations
        b_1  : the first element in rotated Kollo skewness
        k_11 : the first element in rotated Kollo kurtosis
        Data : the rotated sample 
        
        Return
        ----------
        L_1    : the first column for a scaled L_star
        '''  
        Ones = np.ones((m, 1))
        dof = 6 # the number of orthogonal vectors need; this is a heuristic input, best at 6. cannot be lower than 4. (see paper page 14)
        
        flag = 0
        while flag == 0:
            # generate orthonormal vectors using bootstrapping method
            v1 = np.array(random.choices(Data, k = dof * m)).reshape((m, dof))
            v = np.concatenate((Ones, v1), 1)
            
            # orthogonalise these vectors
            v = self.GS_k(v, 1)
            Z = v[:, 1:]

            # define the equations for the first column 
            def f1(x):
                y = np.zeros((dof, 1))   # y is a column vector
                y[0] = np.sum(x**2) - 1
                y[1] = np.mean(np.dot(Z, x)**3) - b_1
                y[2] = np.mean(np.dot(Z, x)**4) - k11
                return y
            
            # solve equations to get the coefficients
            x_init = np.zeros((dof, 1))
            x_init[0] = 0.9 # initial value, heuristically 0.9 seems to be easier for root() functions to find a solution
            x_ret = root(f1, x_init, method = 'broyden1') 
            if x_ret.success:
                flag = 1
                
        # obtain the solution of the coefficents
        c = x_ret.x
        
        # obtain the first column of scaled L matrix
        L_1 = np.dot(Z, c)

        return L_1    

###################################################################

    def RandomOmega(self, n, fixedOmega):    
        '''
        This function is to create a random rotation matrix based on normal random vectors
        
        Parameters
        ----------
        m: the size of rotation matrix 
        fixedOmega : using the fixed rotation matrix in the paper if True, or using the random normal rotation matrix 
        
        Return
        -------
        omega: rotation matrix based on normal random vectors
        '''
        l = np.ones([n, 1]) / math.sqrt(n)  # set the first column of the rotation matrix
        
        if fixedOmega:     # the fixed roation matrix (see paper page 9)
            R = np.zeros((n - 1, n))
            for i in range(0, n - 1):
                R[i, 0:(i+1)] = (-1)**i / math.sqrt((i + 1) * (i + 2))
                R[i, i+1] = (-1)**(i + 1) * (i + 1) / math.sqrt((i + 1) * (i + 2))
            R = R.T    
            omega = np.concatenate((l, R), axis = 1)
            
        else:   # random roation matrix based on normal random vectors
            # generate random vectors
            mean, cov = np.zeros(n - 1), np.identity(n - 1) 
            R = np.random.multivariate_normal(mean, cov, n)     # column 2 to m
            v = np.concatenate((l, R), axis = 1)
            
            # orthogonalise them to form a rotation matrix
            omega = self.GS_k(v, 1)
            for i in range (1, n):
                omega[:,i] = omega[:,i] / np.sqrt(np.dot(omega[:,i].T, omega[:,i]))
        
        return omega

    def GS_k(self, v, k): 
        '''
        This function is to conduct G-S orthogonalization process starting from the (k+1)th column
        
        Parameters
        ----------
        v : the matrix given to do G-S process with its the first k columns being orthogonalized 
        k  : process start from k+1 column, as the first k columns have been orthogonalized 
         
        Return
        -------
        u : the matrix after orthonormalization
        '''         
        (m, n) = np.shape(v)
        u = np.zeros((m, n))
        u[:, 0:k] = v[:, 0:k]    # obtain the orthogonal part in v
     
        for l in range(k, n):   # starting from k+1 column, orthogonalise the remaining columns in u
            projsum = np.zeros((1, m))
            for i in range(0,l):
                projsum += np.dot(u[:, i].T, v[:, l]) / np.dot(u[:, i].T, u[:, i])* u[:, i]
            u[:, l] = v[:, l] - projsum
            var = np.sqrt(np.dot(u[:, l].T, u[:, l]))
            u[:, l] = np.sqrt(m) * u[:, l] / var

        return u

###################################################################
############ Kollo skewness and kurtosis by definition ############
###################################################################
    def Kolloskewness(self, L):
        '''
        This function is to obtain the Kollo skewenss from a standarised data
        
        Parameters
        ----------
        L: the standardised data also the L matrix in ROM 
        
        Return
        -------
        Skew: the Kollo skewness 
        '''
        
        (m, n) = L.shape
        unit = np.ones((n,1))
        temp = np.array([(L[k,:].dot(unit))**2 * L[k,:]   for k in range(0, m) ]).sum(axis = 0)
        Skew = ( m**(-1) ) * temp
        
        Skew = Skew.reshape((n,1))
        
        return Skew
    
    def Kollokurtosis(self, L):
        '''
        This function is to obtain the Kollo kurtosis from a standarised data
        
        Parameters
        ----------
        L: the standardised data also the L matrix in ROM 
        
        Return
        -------
        Kurtosis: the Kollo kurtosis 
        '''
        (m, n) = L.shape
        temp, unit = np.zeros((n, n)), np.ones((n, 1))
        for k in range(0, m):
            a = L[k,:].reshape((n, 1))
            temp += (L[k,:].dot(unit))**2 * a.dot(a.T) 
        Kurtosis = ( m**(-1) ) * temp
        
        return Kurtosis